# Informations sur les institutions culturelles en France

## Des lieux différents répartis sur le territoire

Pour ce notebook, nous allons utiliser la librairie python "folium" qui permet d'afficher facilement des carte
<br>Nous utilisons également la librairie python "pandas" pour manipuler les différents jeux de données
<br>Nous utilisons également la librairie python "geopandas" qui nous permet de manipuler des fichiers contenant des données géographiques

In [18]:
import folium
import pandas as pd
import geopandas as gpd

On initie la carte folium avec un point central (trouvé ici : https://fr.wikipedia.org/wiki/Centre_de_la_France) et un zoom de départ permettant de visualiser l'ensemble du pays. La doc folium se trouve ici : https://python-visualization.github.io/folium/

In [19]:
m = folium.Map(location=[46.759281,2.401860], tiles="OpenStreetMap", zoom_start=6)

On charge un fichier csv open data (données du ministère de la culture) Lien : https://data.culture.gouv.fr/explore/dataset/liste-des-structures-du-spectacle-vivant-subventionnees-par-le-ministere/information/

In [20]:
data = pd.read_csv('liste-des-structures-du-spectacle-vivant-subventionnees-par-le-ministere.csv', error_bad_lines=False, sep=";")

On en affiche un extrait

In [21]:
data

,Structure,NOM1,NOM2,ADRESSE1,ADRESSE2,CP,VILLE,Longitude,Latitude,coordonnees_geoloc,coordonnees_ban,coordonnees_finales,Région
0,Opéra en région,Opéra National de Lyon,NaN,1 place de la Comédie,NaN,66203.0,LYON Cedex,4.837869,45.768140,"45.76813999999999,4.837869",NaN,"45.76814, 4.837869",Auvergne-Rhône-Alpes
1,Opéra en région,Opéra de Marseille,NaN,2 rue Molière,NaN,13001.0,MARSEILLE,5.375878,43.293935,"43.293934999999976,5.375878000000001","43.293839, 5.376004","43.293935, 5.375878",Provence-Alpes-Côte d'Azur
2,Opéra en région,Angers-Nantes Opera (Ano),NaN,1 rue Molière,NaN,44000.0,NANTES,-1.561901,47.214005,"47.21400499999999,-1.561901","47.213652, -1.561991","47.214005, -1.561901",Pays de la Loire
3,Opéra en région,Opéra National du Rhin,NaN,19 place de la Broglie,B.P. 320,67008.0,STRASBOURG,7.749568,48.584960,"48.58496000000001,7.749568",NaN,"48.58496, 7.749568",Grand Est
4,Opéra en région,Opéra de Rouen/Haute Normandie,NaN,7 rue du Docteur Rambert,B.P. 1253,76177.0,ROUEN Cedex,1.089818,49.439491,"49.439491000000004,1.089818",NaN,"49.439491, 1.089818",Normandie
5,Opéra en région,Opéra de Tours,NaN,34 rue de la Sellerie,NaN,37000.0,TOURS,0.690005,47.394378,"47.394377999999996,0.690005","47.394422, 0.689971","47.394378, 0.690005",Centre-Val de Loire
6,Centre chorégraphique national,Centre chorégraphique national de Créteil et d...,NaN,Maison des Arts Place Salvador Allende,NaN,94000.0,Créteil,2.457911,48.800463,"48.800463,2.4579109999999997",NaN,"48.800463, 2.457911",Île-de-France
7,Centre chorégraphique national,Centre chorégraphique national à La Rochelle,NaN,Chapelle Fromentin,"14, rue du collège",17025.0,La Rochelle cedex 1,-1.173110,46.159895,"46.159895,-1.17311",NaN,"46.159895, -1.17311",Nouvelle-Aquitaine
8,Centre chorégraphique national,Ballet national de Marseille,NaN,"20, bd de Gabès",NaN,13417.0,Marseille Cedex 08,5.383504,43.265056,"43.265056,5.383504",NaN,"43.265056, 5.383504",Provence-Alpes-Côte d'Azur
9,Centre chorégraphique national,Centre chorégraphique national de Montpellier ...,NaN,Les Ursulines,Boulevard Louis Blanc,34000.0,Montpellier,3.878514,43.613803,"43.613803,3.878514","43.611024, 3.875521","43.613803, 3.878514",Occitanie


On se débarasse des lignes n'ayant pas de valeur sur certaines colonnes (car on ne peut rien en faire)

In [22]:
data = data.dropna(subset=['Longitude', 'Latitude', 'Région'])

On détermine une couleur pour chaque type de structure (pourrait être optimiser)

In [23]:
colors = {
    'Opéra en région':'red',
    'Centre chorégraphique national':'blue',
    'Orchestre Permanent':'lightblue',
    'Centre national des arts de la rue':'purple',
    'Centres nationaux de création musicale':'green',
    'Scène Nationale':'orange',
    'Centre Dramatique National':'pink',
    'Centre Dramatique Régional':'lightred'
}

On ajoute cette information (couleur) dans une nouvelle colonne au dataframe principal

In [24]:
data['color'] = data['Structure'].apply(lambda x:colors[x])

C:\Users\Geof\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Nous créons des marqueurs sur la carte pour chaque structure. Pour y arriver, on parcours tout le dataframe ligne par ligne

In [25]:
for index, row in data.iterrows():
    folium.Marker([row['Latitude'],row['Longitude']], popup='<i>'+row['NOM1']+'<br/>'+row['ADRESSE1']+'</i>', tooltip=row['NOM1'], icon=folium.Icon(color=row['color'])).add_to(m)

On affiche le résultat

In [26]:
m

## Une inégalité de structures en fonction des régions

On regarde combien de structures par région

In [27]:
data['Région'].value_counts()

Auvergne-Rhône-Alpes          21
Île-de-France                 20
Grand Est                     20
Occitanie                     16
Nouvelle-Aquitaine            15
Normandie                     14
Provence-Alpes-Côte d'Azur    14
Hauts-de-France               14
Bourgogne-Franche-Comté       10
Pays de la Loire              10
Bretagne                       9
Centre-Val de Loire            9
Name: Région, dtype: int64

On transforme cela en dataframe

In [28]:
stat_regions = pd.DataFrame(data['Région'].value_counts().reset_index().values, columns=["Region", "Nb"])

In [29]:
stat_regions.head(12)

,Region,Nb
0,Auvergne-Rhône-Alpes,21
1,Île-de-France,20
2,Grand Est,20
3,Occitanie,16
4,Nouvelle-Aquitaine,15
5,Normandie,14
6,Provence-Alpes-Côte d'Azur,14
7,Hauts-de-France,14
8,Bourgogne-Franche-Comté,10
9,Pays de la Loire,10


On charge un fichier geojson représentant le contour des différentes régions de France. Fichier récupéré ici : https://github.com/gregoiredavid/france-geojson

In [30]:
regions = gpd.read_file('regions-version-simplifiee.geojson')

On l'affiche, il contient la description géographique pour chacune des régions

In [31]:
regions

,code,nom,geometry
0,11,Île-de-France,"POLYGON ((2.590524279394622 49.07965484673242,..."
1,24,Centre-Val de Loire,"POLYGON ((2.874625206405681 47.52042311253978,..."
2,27,Bourgogne-Franche-Comté,"POLYGON ((3.629423683496518 46.74945902193754,..."
3,28,Normandie,"POLYGON ((-1.119622540266051 49.3555679898469,..."
4,32,Hauts-de-France,"POLYGON ((4.047972977986833 49.40564229475753,..."
5,44,Grand Est,"POLYGON ((4.233164134987158 49.95775092391093,..."
6,52,Pays de la Loire,(POLYGON ((-2.45848916517333 47.44812172071826...
7,53,Bretagne,(POLYGON ((-3.659143819670574 48.6592097806692...
8,75,Nouvelle-Aquitaine,(POLYGON ((1.415190510113751 46.34721849587296...
9,76,Occitanie,(POLYGON ((1.786125011063897 42.57362343207788...


On créé une seconde carte pour nos besoins, toujours centré au même endroit

In [32]:
m2 = folium.Map(location=[46.759281,2.401860], tiles="OpenStreetMap", zoom_start=6)

On créé une carte choroplèthe (voir ici : https://fr.wikipedia.org/wiki/Carte_choropl%C3%A8the) 
<br>On fait un join entre les données géographiques et le dataframe contenant le nombre de structures par régions (avec un identifiant unique qui est le nom de la région). On fait un dégradé de couleur pour afficher les différences entre régions

In [33]:
m2.choropleth(
    geo_data=regions,
    name="Statistiques",
    data=stat_regions,
    columns=['Region','Nb'],
    key_on='feature.properties.nom',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Statistiques Régions"
    )

On affiche la seconde carte

In [34]:
m2